# Forecasting Procedure

### Import packages and load file

In [1]:
import pandas as pd
from pandas import read_excel
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

pd.set_option('Display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
import asgl 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn_timeseries import TimeSeriesRandomForestRegressor
from sklearn_timeseries import optimal_block_length
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_raw = read_excel("./data_womacro_test.xlsx", index_col = 0)
df_raw = df_raw.rename(columns=lambda x: x[1:] if x.startswith('^') else x)

""" Below this is all robustness check arguments """
# df_raw = read_excel("./data_update_0515.xlsx", index_col = 0)
# df_raw = df_raw.rename(columns=lambda x: x[1:] if x.startswith('^') else x)

# df_raw = df_raw.loc[:,~(df_raw.columns.str.contains("google"))]
# google_df = pd.read_csv("../google_trend_update_short_0516.csv")
# google_df['date'] = pd.to_datetime(google_df['date'].astype("string")).dt.to_period('d')
# google_df = google_df.set_index('date').to_timestamp()
# google_df.columns = ['google_1', 'google_2', 'google_3', 'google_4', 'google_5', 'google_6']
# df_raw = pd.concat([df_raw, google_df.loc[google_df.index.isin(df_raw.index)]], axis=1)

# epu = pd.read_csv("../epu_news.csv")
# epu['date'] = pd.to_datetime(epu['date'].astype("string")).dt.to_period('d')
# epu = epu.set_index('date').to_timestamp()
# df_raw = pd.concat([df_raw, epu.loc[epu.index.isin(df_raw.index)]], axis=1)

# df_raw = df_raw.iloc[1459:2461,:] # 2016-01-04 ~ 2020-01-31

df_raw = df_raw.loc[(df_raw.index > '2010-01-01') & (df_raw.index < '2018-12-31')]
# df_raw = df_raw.loc[(df_raw.index > '2018-12-31') & (df_raw.index < '2022-12-31')]

### Model Forecasting (Rolling Window)

In [12]:
def runfc(Y,indice,lag,method,selection,function) :

    y_scaler = MinMaxScaler()
    scaled = y_scaler.fit_transform(Y.iloc[:,indice].values.reshape(-1,1)) # For inverse transformation    
    scaled = pd.DataFrame(MinMaxScaler().fit_transform(Y.values), columns = Y.columns)
    Y2 = scaled

    list_of_df = [] # making list of df

    """ CODE FOR ARRANGING COLUMN NAMES """
    for j, names in enumerate(range(lag+4)) :
        df_temp = Y2.shift(j)
        df_temp.columns = [col_name+'_lag{}'.format(j) for col_name in df_temp.columns] # warning (feature name must match)
        list_of_df.append(df_temp)  
        
    aux = pd.concat(list_of_df, axis=1).dropna()
        
    y = aux.iloc[:,indice]
    X = aux.iloc[:,Y2.shape[1]*lag:]

    if lag == 1 :
        X_out = aux.tail(1).iloc[:,0:X.shape[1]]
        k =1
            
    else :
        X_out = aux.iloc[:,Y2.shape[1]*(lag-1):]
        X_out = X_out.tail(1).iloc[:,0:X.shape[1]]

    X = X.loc[:,(~(X.columns.str.contains('VIX')) | (X.columns.isin(X.loc[:,X.columns.str.contains('VIX')].iloc[:,0:3].columns)))] # Only VIX with first 3
    X_out = X_out.loc[:,(~(X_out.columns.str.contains('VIX')) | (X_out.columns.isin(X_out.loc[:,X_out.columns.str.contains('VIX')].iloc[:,0:3].columns)))] # Only VIX with first 3


    """ METHOD : HAR / ECO / SENT """

    if method == 'HAR':
        X = X.loc[:, X.columns.str.contains("VIX")]
        X_out = X_out.loc[:, X_out.columns.str.contains("VIX")]

    if method == 'ECO':
        X = X.loc[:, ~(X.columns.str.contains("dis") | X.columns.str.contains("Score") | X.columns.str.contains("google_1") | 
            X.columns.str.contains("google_2") | X.columns.str.contains("google_3") | X.columns.str.contains("google_4") | 
            X.columns.str.contains("google_5") | X.columns.str.contains("google_6") | X.columns.str.contains("policy"))]
        
        X_out = X_out.loc[:, ~(X_out.columns.str.contains("dis") | X_out.columns.str.contains("Score") | X_out.columns.str.contains("google_1") | 
            X_out.columns.str.contains("google_2") | X_out.columns.str.contains("google_3") | X_out.columns.str.contains("google_4") | 
            X_out.columns.str.contains("google_5") | X_out.columns.str.contains("google_6") | X_out.columns.str.contains("policy"))]

    if method =='SENT':
        pass
    
    else : 
        pass


    if lag == 5 : k=1
    
    if (lag == 10) or (lag == 22) : k=2

    """ Variable Selection when ITERATION """

    if selection == 'LASSO':
        lasso_X = X.loc[:,(~(X.columns.str.contains("VIX")))]
        kfcv = KFold(n_splits=5, shuffle=False)
        search_kfcv = GridSearchCV(Lasso(), {'alpha':np.arange(0.001,2,0.1)}, cv=kfcv, scoring='neg_mean_squared_error', verbose=0)
        search_kfcv.fit(lasso_X.values,y)
        best_param = search_kfcv.best_params_ # SAVE
        
        coefficients = search_kfcv.best_estimator_.coef_
        lasso_importances = pd.Series(np.abs(coefficients), index = lasso_X.columns)
        lasso_importances_top = lasso_importances.loc[lasso_importances != 0].sort_values(ascending=False)
        
        X = X.loc[:,(X.columns.isin(lasso_importances_top.index)) | (X.columns.str.contains("VIX"))]
        X_out = X_out.loc[:,(X_out.columns.isin([s[:-k] + str(int(s[-k:])-1) for s in lasso_importances_top.index])) 
                        | (X_out.columns.str.contains("VIX"))]

    if selection == 'ALASSO':
        """ Adaptive Lasso """
        lasso_X = X.loc[:,(~(X.columns.str.contains("VIX")))]
        
        # lambda_list =np.arange(0.001,2,0.1)
        # # mdl_cv = asgl.CV(model='lm',penalization='alasso',lambda1 = lambda_list,weight_technique='unpenalized',random_state=42, nfolds=5)
        # # lambda1 = lambda_list[mdl_cv.cross_validation(X.to_numpy(), y.to_numpy()).mean(axis=1).argmin()]
        
        lambda1 = 0.001 # Optimal lambda for almost all (for computational time we set this to 0.001)
        weights = asgl.WEIGHTS(model='lm', penalization="alasso", weight_technique='unpenalized')
        lasso_weights,_ = weights.fit(lasso_X.to_numpy(), y.to_numpy())
        mdl_alasso = asgl.ASGL(model='lm', penalization='alasso', lambda1 = lambda1, lasso_weights=lasso_weights, parallel=True, intercept=False)
        mdl_alasso.fit(lasso_X.to_numpy(), y.to_numpy())
        
        coefficients = mdl_alasso.coef_
        lasso_importances = pd.Series(np.abs(coefficients[0]), index = lasso_X.columns)
        lasso_importances_top = lasso_importances.loc[lasso_importances != 0].sort_values(ascending=False)

        X = X.loc[:,(X.columns.isin(lasso_importances_top.index)) | (X.columns.str.contains("VIX"))]
        X_out = X_out.loc[:,(X_out.columns.isin([s[:-k] + str(int(s[-k:])-1) for s in lasso_importances_top.index])) 
                        | (X_out.columns.str.contains("VIX"))]
    
    if selection == 'RF_IID' :
        
        """ Random Forest with IID """
        lasso_X = X.loc[:,(~(X.columns.str.contains("VIX")))]
        
        # kfcv = KFold(n_splits=5, shuffle=False)
        # parameters = {'n_estimators': [100, 150, 200],'max_depth': [4,6,8]}
        # grid_cv = GridSearchCV(TimeSeriesRandomForestRegressor(bootstrap='iid', random_state=42, n_jobs = -1), parameters, cv = kfcv, n_jobs= -1)
        # grid_cv.fit(lasso_X.values, y)
        
        # rf_run = TimeSeriesRandomForestRegressor(bootstrap='iid', random_state=42, max_depth=grid_cv.best_params_['max_depth'], n_estimators=grid_cv.best_params_['n_estimators']) # gridsearch best parametesr
        rf_run = TimeSeriesRandomForestRegressor(bootstrap='iid', random_state=42, max_depth=8, n_estimators=150)
        rf_run.fit(lasso_X.values, y)
        rf_importances_values = rf_run.feature_importances_
        rf_importances = pd.Series(rf_importances_values, index = lasso_X.columns)
        rf_importances_top = rf_importances.sort_values(ascending=False)[0:20]
        
        X = X.loc[:,(X.columns.isin(rf_importances_top.index)) | (X.columns.str.contains("VIX"))]
        X_out = X_out.loc[:,(X_out.columns.isin([s[:-k] + str(int(s[-k:])-1) for s in rf_importances_top.index])) 
                        | (X_out.columns.str.contains("VIX")) ]
        
    # if selection == 'RF_SB' :
        
    #     """ Random Forest with IID """
    #     lasso_X = X.loc[:,(~(X.columns.str.contains("VIX")))]
        
    #     # kfcv = KFold(n_splits=5, shuffle=False)
    #     # parameters = {'n_estimators': [100, 150, 200],'max_depth': [4,6,8]}
    #     # grid_cv = GridSearchCV(TimeSeriesRandomForestRegressor(bootstrap='sb', random_state=42, n_jobs = -1), parameters, cv = kfcv, n_jobs= -1)
    #     # grid_cv.fit(lasso_X.values, y)
        
    #     # rf_run = TimeSeriesRandomForestRegressor(bootstrap='sb', random_state=42, max_depth=grid_cv.best_params_['max_depth'], n_estimators=grid_cv.best_params_['n_estimators'])
    #     rf_run = TimeSeriesRandomForestRegressor(bootstrap='sb', random_state=42, max_depth=6, n_estimators=100) # gridsearch best parametesr
    #     rf_run.fit(lasso_X.values, y)
    #     rf_importances_values = rf_run.feature_importances_
    #     rf_importances = pd.Series(rf_importances_values, index = lasso_X.columns)
    #     rf_importances_top = rf_importances.sort_values(ascending=False)[0:20]
        
        
    #     fig = plt.figure(figsize=(8,6))
    #     sns.barplot(x=rf_importances_top, y=rf_importances_top.index)
    #     plt.xlabel("Importance")
    #     plt.ylabel("Features")
    #     plt.title("Feature Importance : IID Bootstrap")
    #     plt.close()

    #     X = X.loc[:,(X.columns.isin(rf_importances_top.index)) | (X.columns.str.contains("VIX"))]
    #     X_out = X_out.loc[:,(X_out.columns.isin([s[:-2] + str(int(s[-2:])-1) for s in rf_importances_top.index])) 
    #                     | (X_out.columns.str.contains("VIX"))]

    else : 
        pass
    
    """ FUNCTION """

    if function == 'OLS':
        model = LinearRegression().fit(X.values,y)
        pred = y_scaler.inverse_transform(model.predict(X_out.values).reshape(-1,1))

    if function == 'RF_IID':
        model = TimeSeriesRandomForestRegressor(bootstrap='iid', random_state=42, max_depth=8, n_estimators=150, n_jobs = -1).fit(X.values,y)
        pred = y_scaler.inverse_transform(model.predict(X_out.values).reshape(-1,1))

    # if function == "RF_SB" :
    #     model = TimeSeriesRandomForestRegressor(bootstrap='sb', random_state=42, max_depth=6, n_estimators=100, n_jobs = -1).fit(X.values,y)
    #     pred = y_scaler.inverse_transform(model.predict(X_out.values).reshape(-1,1))

    if function == 'XGB' :
        model = XGBRegressor(max_depth=4, n_estimators=100, random_state=42, n_jobs = -1).fit(X.values,y)
        pred = y_scaler.inverse_transform(model.predict(X_out.values).reshape(-1,1)).astype(np.float64)
        
    else : 
        pass
    

    return({"pred":pred, "model":model, "columns":X.columns})

In [13]:
# Selection =  LASSO, ALASSO, RF_IID, RF_SB
# function =  OLS, RF_IID, RF_SB, XGB
ex1 = runfc(Y, indice=0, lag=10, method='SENT', selection='None', function='OLS')
ex1['pred'][0]

array([3.03699229])

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [15]:
def fc_rolling_window(Y, npred, indice=0, lag=None, method=None, selection=None, function=None) :
    
    save_importance = []
    save_pred = pd.DataFrame(np.repeat(0,npred,axis=0))
    
    window_sequence = list(range(npred,0,-1))
    
    for i in window_sequence :
        Y_window = Y.iloc[(npred-i):(Y.shape[0]-i),:]
        runfunciton = runfc(Y_window, indice=0, lag=lag, method=method, selection=selection, function=function)
        save_pred.loc[npred-i] = runfunciton['pred'][0]
        if function != 'OLS':
            save_importance.append(runfunciton['model'].feature_importances_)
        # if i%30 == 0: print("Iteration is working ... Up to {} times".format((npred-i+1)))
    
    real = Y.iloc[:,indice] 

    # predictions = np.append(np.repeat(np.nan, (real.shape[0]-npred)), save_pred)
    # plt.plot(real)
    # plt.plot(predictions)
    # plt.legend(['True','Pred'])
    # plt.close() # dont show plot
    
    mse = mean_squared_error(real.tail(npred), save_pred.squeeze())
    mae = mean_absolute_error(real.tail(npred), save_pred.squeeze())
    errors = pd.DataFrame({'mse':mse, 'mae':mae}, index=['errors']) 

    return({"pred":save_pred, "errors":errors, "save_importantce":save_importance, "columns":runfunciton['columns']})

In [16]:
new_names = ['sent_none','eco_none','har_none','sent_lasso','eco_lasso','sent_alasso','eco_alasso','sent_rfiid','eco_rfiid']

#### 1. OLS

In [17]:
ols_sent_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection=None, function='OLS')
ols_eco_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection=None, function='OLS')
ols_har_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection=None, function='OLS')

In [18]:
ols_har_none_1['errors'], ols_eco_none_1['errors'], ols_sent_none_1['errors']

(           mse     mae
 errors 0.00492 0.05043,
            mse     mae
 errors 0.01049 0.08291,
            mse     mae
 errors 0.00965 0.07856)

In [19]:
""" OLS with no selection"""
ols_sent_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection=None, function='OLS')
ols_eco_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection=None, function='OLS')
ols_har_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection=None, function='OLS')

""" OLS with LASSO selection"""
ols_sent_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='LASSO', function='OLS')
ols_eco_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='LASSO', function='OLS')
# ols_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='OLS')

""" OLS with ALASSO selection"""
ols_sent_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='ALASSO', function='OLS')
ols_eco_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='ALASSO', function='OLS')
# ols_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='OLS')

""" OLS with RF_IID selection"""
ols_sent_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_IID', function='OLS')
ols_eco_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_IID', function='OLS')
# ols_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='OLS')

# """ OLS with RF_SB selection"""
# ols_sent_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_SB', function='OLS')
# ols_eco_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_SB', function='OLS')
# ols_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='OLS')

In [239]:
# ols_results_1 = pd.concat([ols_sent_none_1['errors'], ols_eco_none_1['errors'], ols_har_none_1['errors'], ols_sent_lasso_1['errors'], ols_eco_lasso_1['errors'],
#                            ols_sent_alasso_1['errors'], ols_eco_alasso_1['errors'], ols_sent_rfiid_1['errors'], ols_eco_rfiid_1['errors'],  ols_sent_rfsb_1['errors'],
#                            ols_eco_rfsb_1['errors']], axis = 0).transpose()
# ols_results_1.columns = new_names
# ols_results_1

In [20]:
ols_results_1 = pd.concat([ols_sent_none_1['errors'], ols_eco_none_1['errors'], ols_har_none_1['errors'], ols_sent_lasso_1['errors'], ols_eco_lasso_1['errors'],
                           ols_sent_alasso_1['errors'], ols_eco_alasso_1['errors'], ols_sent_rfiid_1['errors'], ols_eco_rfiid_1['errors']], axis = 0).transpose()
ols_results_1.columns = new_names
ols_results_1

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.00965,0.01049,0.00492,0.00452,0.00555,0.00452,0.00543,0.00459,0.00551
mae,0.07856,0.08291,0.05043,0.04945,0.05711,0.05039,0.05611,0.04884,0.05688


In [17]:
# ols_predictions_1 = pd.concat([ols_sent_none_1['pred'], ols_eco_none_1['pred'], ols_har_none_1['pred'], ols_sent_lasso_1['pred'], ols_eco_lasso_1['pred'],
#                            ols_sent_alasso_1['pred'], ols_eco_alasso_1['pred'], ols_sent_rfiid_1['pred'], ols_eco_rfiid_1['pred'],  ols_sent_rfsb_1['pred'],
#                            ols_eco_rfsb_1['pred']], axis = 1)
# ols_predictions_1.columns = new_names
# ols_predictions_1

In [209]:
# ols_predictions_1 = pd.concat([ols_sent_none_1['pred'], ols_eco_none_1['pred'], ols_har_none_1['pred'], ols_sent_lasso_1['pred'], ols_eco_lasso_1['pred'],
#                            ols_sent_alasso_1['pred'], ols_eco_alasso_1['pred'], ols_sent_rfiid_1['pred'], ols_eco_rfiid_1['pred']], axis = 1)
# ols_predictions_1.columns = new_names
# ols_predictions_1

In [21]:
""" LAG = 5 """

""" OLS with no selection"""
ols_sent_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection=None, function='OLS')
ols_eco_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection=None, function='OLS')
ols_har_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='HAR', selection=None, function='OLS')

""" OLS with LASSO selection"""
ols_sent_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='LASSO', function='OLS')
ols_eco_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='LASSO', function='OLS')
# ols_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='OLS')

""" OLS with ALASSO selection"""
ols_sent_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='ALASSO', function='OLS')
ols_eco_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='ALASSO', function='OLS')
# ols_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='OLS')

""" OLS with RF_IID selection"""
ols_sent_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_IID', function='OLS')
ols_eco_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_IID', function='OLS')
# ols_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='OLS')

# """ OLS with RF_SB selection"""
# ols_sent_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_SB', function='OLS')
# ols_eco_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_SB', function='OLS')
# ols_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='OLS')

In [65]:
# ols_results_5 = pd.concat([ols_sent_none_5['errors'], ols_eco_none_5['errors'], ols_har_none_5['errors'], ols_sent_lasso_5['errors'], ols_eco_lasso_5['errors'],
#                            ols_sent_alasso_5['errors'], ols_eco_alasso_5['errors'], ols_sent_rfiid_5['errors'], ols_eco_rfiid_5['errors'],  ols_sent_rfsb_5['errors'],
#                            ols_eco_rfsb_5['errors']], axis = 0).transpose()
# ols_results_5.columns = new_names
# ols_results_5

In [22]:
ols_results_5 = pd.concat([ols_sent_none_5['errors'], ols_eco_none_5['errors'], ols_har_none_5['errors'], ols_sent_lasso_5['errors'], ols_eco_lasso_5['errors'],
                           ols_sent_alasso_5['errors'], ols_eco_alasso_5['errors'], ols_sent_rfiid_5['errors'], ols_eco_rfiid_5['errors']], axis = 0).transpose()
ols_results_5.columns = new_names
ols_results_5

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.03706,0.03688,0.01802,0.02644,0.02466,0.02159,0.02346,0.02158,0.02015
mae,0.14315,0.14936,0.10113,0.12896,0.12829,0.11524,0.12194,0.10817,0.11553


In [67]:
# ols_predictions_5 = pd.concat([ols_sent_none_5['pred'], ols_eco_none_5['pred'], ols_har_none_5['pred'], ols_sent_lasso_5['pred'], ols_eco_lasso_5['pred'],
#                            ols_sent_alasso_5['pred'], ols_eco_alasso_5['pred'], ols_sent_rfiid_5['pred'], ols_eco_rfiid_5['pred'],  ols_sent_rfsb_5['pred'],
#                            ols_eco_rfsb_5['pred']], axis = 1)
# ols_predictions_5.columns = new_names
# ols_predictions_5

In [ ]:
# ols_predictions_5 = pd.concat([ols_sent_none_5['pred'], ols_eco_none_5['pred'], ols_har_none_5['pred'], ols_sent_lasso_5['pred'], ols_eco_lasso_5['pred'],
#                            ols_sent_alasso_5['pred'], ols_eco_alasso_5['pred'], ols_sent_rfiid_5['pred'], ols_eco_rfiid_5['pred']], axis = 1)
# ols_predictions_5.columns = new_names
# ols_predictions_5

In [23]:
""" LAG = 10 """

""" OLS with no selection"""
ols_sent_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection=None, function='OLS')
ols_eco_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection=None, function='OLS')
ols_har_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='HAR', selection=None, function='OLS')

""" OLS with LASSO selection"""
ols_sent_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='LASSO', function='OLS')
ols_eco_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='LASSO', function='OLS')
# ols_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='OLS')

""" OLS with ALASSO selection"""
ols_sent_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='ALASSO', function='OLS')
ols_eco_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='ALASSO', function='OLS')
# ols_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='OLS')

""" OLS with RF_IID selection"""
ols_sent_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_IID', function='OLS')
ols_eco_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_IID', function='OLS')
# ols_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='OLS')

# """ OLS with RF_SB selection"""
# ols_sent_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_SB', function='OLS')
# ols_eco_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_SB', function='OLS')
# ols_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='OLS')

In [70]:
# ols_results_10 = pd.concat([ols_sent_none_10['errors'], ols_eco_none_10['errors'], ols_har_none_10['errors'], ols_sent_lasso_10['errors'], ols_eco_lasso_10['errors'],
#                            ols_sent_alasso_10['errors'], ols_eco_alasso_10['errors'], ols_sent_rfiid_10['errors'], ols_eco_rfiid_10['errors'],  ols_sent_rfsb_10['errors'],
#                            ols_eco_rfsb_10['errors']], axis = 0).transpose()
# ols_results_10.columns = new_names
# ols_results_10


In [24]:
ols_results_10 = pd.concat([ols_sent_none_10['errors'], ols_eco_none_10['errors'], ols_har_none_10['errors'], ols_sent_lasso_10['errors'], ols_eco_lasso_10['errors'],
                           ols_sent_alasso_10['errors'], ols_eco_alasso_10['errors'], ols_sent_rfiid_10['errors'], ols_eco_rfiid_10['errors']], axis = 0).transpose()
ols_results_10.columns = new_names
ols_results_10

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.05643,0.04591,0.02874,0.04074,0.03752,0.03584,0.03930,0.04077,0.03587
mae,0.18964,0.17286,0.13266,0.16258,0.15801,0.15585,0.16157,0.17392,0.16154


In [72]:
# ols_predictions_10 = pd.concat([ols_sent_none_10['pred'], ols_eco_none_10['pred'], ols_har_none_10['pred'], ols_sent_lasso_10['pred'], ols_eco_lasso_10['pred'],
#                            ols_sent_alasso_10['pred'], ols_eco_alasso_10['pred'], ols_sent_rfiid_10['pred'], ols_eco_rfiid_10['pred'],  ols_sent_rfsb_10['pred'],
#                            ols_eco_rfsb_10['pred']], axis = 1)
# ols_predictions_10.columns = new_names
# ols_predictions_10

In [ ]:
# ols_predictions_10 = pd.concat([ols_sent_none_10['pred'], ols_eco_none_10['pred'], ols_har_none_10['pred'], ols_sent_lasso_10['pred'], ols_eco_lasso_10['pred'],
#                            ols_sent_alasso_10['pred'], ols_eco_alasso_10['pred'], ols_sent_rfiid_10['pred'], ols_eco_rfiid_10['pred']], axis = 1)
# ols_predictions_10.columns = new_names
# ols_predictions_10

In [25]:
""" LAG = 22 """

""" OLS with no selection"""
ols_sent_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection=None, function='OLS')
ols_eco_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection=None, function='OLS')
ols_har_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='HAR', selection=None, function='OLS')

""" OLS with LASSO selection"""
ols_sent_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='LASSO', function='OLS')
ols_eco_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='LASSO', function='OLS')
# ols_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='OLS')

""" OLS with ALASSO selection"""
ols_sent_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='ALASSO', function='OLS')
ols_eco_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='ALASSO', function='OLS')
# ols_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='OLS')

""" OLS with RF_IID selection"""
ols_sent_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_IID', function='OLS')
ols_eco_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_IID', function='OLS')
# ols_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='OLS')

# """ OLS with RF_SB selection"""
# ols_sent_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_SB', function='OLS')
# ols_eco_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_SB', function='OLS')
# ols_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='OLS')

In [75]:
# ols_results_22 = pd.concat([ols_sent_none_22['errors'], ols_eco_none_22['errors'], ols_har_none_22['errors'], ols_sent_lasso_22['errors'], ols_eco_lasso_22['errors'],
#                            ols_sent_alasso_22['errors'], ols_eco_alasso_22['errors'], ols_sent_rfiid_22['errors'], ols_eco_rfiid_22['errors'],  ols_sent_rfsb_22['errors'],
#                            ols_eco_rfsb_22['errors']], axis = 0).transpose()
# ols_results_22.columns = new_names
# ols_results_22

In [26]:
ols_results_22 = pd.concat([ols_sent_none_22['errors'], ols_eco_none_22['errors'], ols_har_none_22['errors'], ols_sent_lasso_22['errors'], ols_eco_lasso_22['errors'],
                           ols_sent_alasso_22['errors'], ols_eco_alasso_22['errors'], ols_sent_rfiid_22['errors'], ols_eco_rfiid_22['errors']], axis = 0).transpose()
ols_results_22.columns = new_names
ols_results_22

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.05499,0.04631,0.03581,0.03334,0.03191,0.03336,0.03094,0.07822,0.07530
mae,0.18056,0.16749,0.15662,0.14748,0.15023,0.14983,0.14668,0.22877,0.21538


In [77]:
# ols_predictions_22 = pd.concat([ols_sent_none_22['pred'], ols_eco_none_22['pred'], ols_har_none_22['pred'], ols_sent_lasso_22['pred'], ols_eco_lasso_22['pred'],
#                            ols_sent_alasso_22['pred'], ols_eco_alasso_22['pred'], ols_sent_rfiid_22['pred'], ols_eco_rfiid_22['pred'],  ols_sent_rfsb_22['pred'],
#                            ols_eco_rfsb_22['pred']], axis = 1)
# ols_predictions_22.columns = new_names
# ols_predictions_22

In [ ]:
# ols_predictions_22 = pd.concat([ols_sent_none_22['pred'], ols_eco_none_22['pred'], ols_har_none_22['pred'], ols_sent_lasso_22['pred'], ols_eco_lasso_22['pred'],
#                            ols_sent_alasso_22['pred'], ols_eco_alasso_22['pred'], ols_sent_rfiid_22['pred'], ols_eco_rfiid_22['pred']], axis = 1)
# ols_predictions_22.columns = new_names
# ols_predictions_22

#### 2. RF_IID

In [27]:
""" RF_IID with no selection"""
rfiid_sent_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection=None, function='RF_IID')
rfiid_eco_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection=None, function='RF_IID')
rfiid_har_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection=None, function='RF_IID')

""" RF_IID with LASSO selection"""
rfiid_sent_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='LASSO', function='RF_IID')
rfiid_eco_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='LASSO', function='RF_IID')
# rfiid_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_IID')

""" RF_IID with ALASSO selection"""
rfiid_sent_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='ALASSO', function='RF_IID')
rfiid_eco_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='ALASSO', function='RF_IID')
# rfiid_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_IID')

""" RF_IID with RF_IID selection"""
rfiid_sent_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_IID', function='RF_IID')
rfiid_eco_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_IID', function='RF_IID')
# rfiid_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_IID')

# """ RF_IID with RF_SB selection"""
# rfiid_sent_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_SB', function='RF_IID')
# rfiid_eco_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_SB', function='RF_IID')
# rfiid_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_IID')

In [28]:
# rfiid_results_1 = pd.concat([rfiid_sent_none_1['errors'], rfiid_eco_none_1['errors'], rfiid_har_none_1['errors'], rfiid_sent_lasso_1['errors'], rfiid_eco_lasso_1['errors'],
#                            rfiid_sent_alasso_1['errors'], rfiid_eco_alasso_1['errors'], rfiid_sent_rfiid_1['errors'], rfiid_eco_rfiid_1['errors'],  rfiid_sent_rfsb_1['errors'],
#                            rfiid_eco_rfsb_1['errors']], axis = 0).transpose()
# rfiid_results_1.columns = new_names
# rfiid_results_1

In [29]:
rfiid_results_1 = pd.concat([rfiid_sent_none_1['errors'], rfiid_eco_none_1['errors'], rfiid_har_none_1['errors'], rfiid_sent_lasso_1['errors'], rfiid_eco_lasso_1['errors'],
                           rfiid_sent_alasso_1['errors'], rfiid_eco_alasso_1['errors'], rfiid_sent_rfiid_1['errors'], rfiid_eco_rfiid_1['errors']], axis = 0).transpose()
rfiid_results_1.columns = new_names
rfiid_results_1

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.00419,0.00543,0.00627,0.00343,0.00527,0.00353,0.00539,0.00354,0.00579
mae,0.04923,0.05516,0.05891,0.04423,0.05613,0.04536,0.05559,0.04691,0.05769


In [22]:
# rfiid_predictions_1 = pd.concat([rfiid_sent_none_1['pred'], rfiid_eco_none_1['pred'], rfiid_har_none_1['pred'], rfiid_sent_lasso_1['pred'], rfiid_eco_lasso_1['pred'],
#                            rfiid_sent_alasso_1['pred'], rfiid_eco_alasso_1['pred'], rfiid_sent_rfiid_1['pred'], rfiid_eco_rfiid_1['pred'],  rfiid_sent_rfsb_1['pred'],
#                            rfiid_eco_rfsb_1['pred']], axis = 1)
# rfiid_predictions_1.columns = new_names
# rfiid_predictions_1

In [ ]:
# rfiid_predictions_1 = pd.concat([rfiid_sent_none_1['pred'], rfiid_eco_none_1['pred'], rfiid_har_none_1['pred'], rfiid_sent_lasso_1['pred'], rfiid_eco_lasso_1['pred'],
#                            rfiid_sent_alasso_1['pred'], rfiid_eco_alasso_1['pred'], rfiid_sent_rfiid_1['pred'], rfiid_eco_rfiid_1['pred']], axis = 1)
# rfiid_predictions_1.columns = new_names
# rfiid_predictions_1

In [30]:
""" LAG = 5 """

""" RF_IID with no selection"""
rfiid_sent_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection=None, function='RF_IID')
rfiid_eco_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection=None, function='RF_IID')
rfiid_har_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='HAR', selection=None, function='RF_IID')

""" RF_IID with LASSO selection"""
rfiid_sent_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='LASSO', function='RF_IID')
rfiid_eco_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='LASSO', function='RF_IID')
# rfiid_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_IID')

""" RF_IID with ALASSO selection"""
rfiid_sent_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='ALASSO', function='RF_IID')
rfiid_eco_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='ALASSO', function='RF_IID')
# rfiid_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_IID')

""" RF_IID with RF_IID selection"""
rfiid_sent_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_IID', function='RF_IID')
rfiid_eco_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_IID', function='RF_IID')
# rfiid_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_IID')

# """ RF_IID with RF_SB selection"""
# rfiid_sent_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_SB', function='RF_IID')
# rfiid_eco_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_SB', function='RF_IID')
# rfiid_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_IID')

In [85]:
# rfiid_results_5 = pd.concat([rfiid_sent_none_5['errors'], rfiid_eco_none_5['errors'], rfiid_har_none_5['errors'], rfiid_sent_lasso_5['errors'], rfiid_eco_lasso_5['errors'],
#                            rfiid_sent_alasso_5['errors'], rfiid_eco_alasso_5['errors'], rfiid_sent_rfiid_5['errors'], rfiid_eco_rfiid_5['errors'],  rfiid_sent_rfsb_5['errors'],
#                            rfiid_eco_rfsb_5['errors']], axis = 0).transpose()
# rfiid_results_5.columns = new_names
# rfiid_results_5

In [31]:
rfiid_results_5 = pd.concat([rfiid_sent_none_5['errors'], rfiid_eco_none_5['errors'], rfiid_har_none_5['errors'], rfiid_sent_lasso_5['errors'], rfiid_eco_lasso_5['errors'],
                           rfiid_sent_alasso_5['errors'], rfiid_eco_alasso_5['errors'], rfiid_sent_rfiid_5['errors'], rfiid_eco_rfiid_5['errors']], axis = 0).transpose()
rfiid_results_5.columns = new_names
rfiid_results_5

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01121,0.01103,0.02153,0.01237,0.01237,0.01030,0.01021,0.00919,0.00876
mae,0.08532,0.08478,0.11488,0.08768,0.08654,0.08010,0.07925,0.07331,0.07311


In [87]:
# rfiid_predictions_5 = pd.concat([rfiid_sent_none_5['pred'], rfiid_eco_none_5['pred'], rfiid_har_none_5['pred'], rfiid_sent_lasso_5['pred'], rfiid_eco_lasso_5['pred'],
#                            rfiid_sent_alasso_5['pred'], rfiid_eco_alasso_5['pred'], rfiid_sent_rfiid_5['pred'], rfiid_eco_rfiid_5['pred'],  rfiid_sent_rfsb_5['pred'],
#                            rfiid_eco_rfsb_5['pred']], axis = 1)
# rfiid_predictions_5.columns = new_names
# rfiid_predictions_5

In [ ]:
# rfiid_results_5 = pd.concat([rfiid_sent_none_5['errors'], rfiid_eco_none_5['errors'], rfiid_har_none_5['errors'], rfiid_sent_lasso_5['errors'], rfiid_eco_lasso_5['errors'],
#                            rfiid_sent_alasso_5['errors'], rfiid_eco_alasso_5['errors'], rfiid_sent_rfiid_5['errors'], rfiid_eco_rfiid_5['errors']], axis = 0).transpose()
# rfiid_results_5.columns = new_names
# rfiid_results_5

In [32]:
""" LAG = 10 """

""" RF_IID with no selection"""
rfiid_sent_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection=None, function='RF_IID')
rfiid_eco_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection=None, function='RF_IID')
rfiid_har_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='HAR', selection=None, function='RF_IID')

""" RF_IID with LASSO selection"""
rfiid_sent_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='LASSO', function='RF_IID')
rfiid_eco_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='LASSO', function='RF_IID')
# rfiid_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_IID')

""" RF_IID with ALASSO selection"""
rfiid_sent_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='ALASSO', function='RF_IID')
rfiid_eco_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='ALASSO', function='RF_IID')
# rfiid_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_IID')

""" RF_IID with RF_IID selection"""
rfiid_sent_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_IID', function='RF_IID')
rfiid_eco_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_IID', function='RF_IID')
# rfiid_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_IID')

# """ RF_IID with RF_SB selection"""
# rfiid_sent_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_SB', function='RF_IID')
# rfiid_eco_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_SB', function='RF_IID')
# rfiid_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_IID')

In [90]:
# rfiid_results_10 = pd.concat([rfiid_sent_none_10['errors'], rfiid_eco_none_10['errors'], rfiid_har_none_10['errors'], rfiid_sent_lasso_10['errors'], rfiid_eco_lasso_10['errors'],
#                            rfiid_sent_alasso_10['errors'], rfiid_eco_alasso_10['errors'], rfiid_sent_rfiid_10['errors'], rfiid_eco_rfiid_10['errors'],  rfiid_sent_rfsb_10['errors'],
#                            rfiid_eco_rfsb_10['errors']], axis = 0).transpose()
# rfiid_results_10.columns = new_names
# rfiid_results_10

In [33]:
rfiid_results_10 = pd.concat([rfiid_sent_none_10['errors'], rfiid_eco_none_10['errors'], rfiid_har_none_10['errors'], rfiid_sent_lasso_10['errors'], rfiid_eco_lasso_10['errors'],
                           rfiid_sent_alasso_10['errors'], rfiid_eco_alasso_10['errors'], rfiid_sent_rfiid_10['errors'], rfiid_eco_rfiid_10['errors']], axis = 0).transpose()
rfiid_results_10.columns = new_names
rfiid_results_10

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01623,0.01739,0.02938,0.01400,0.01622,0.01293,0.01284,0.01215,0.01282
mae,0.10032,0.10165,0.13938,0.09279,0.09863,0.08690,0.08739,0.08426,0.08401


In [92]:
# rfiid_predictions_10 = pd.concat([rfiid_sent_none_10['pred'], rfiid_eco_none_10['pred'], rfiid_har_none_10['pred'], rfiid_sent_lasso_10['pred'], rfiid_eco_lasso_10['pred'],
#                            rfiid_sent_alasso_10['pred'], rfiid_eco_alasso_10['pred'], rfiid_sent_rfiid_10['pred'], rfiid_eco_rfiid_10['pred'],  rfiid_sent_rfsb_10['pred'],
#                            rfiid_eco_rfsb_10['pred']], axis = 1)
# rfiid_predictions_10.columns = new_names
# rfiid_predictions_10

In [ ]:
# rfiid_predictions_10 = pd.concat([rfiid_sent_none_10['pred'], rfiid_eco_none_10['pred'], rfiid_har_none_10['pred'], rfiid_sent_lasso_10['pred'], rfiid_eco_lasso_10['pred'],
#                            rfiid_sent_alasso_10['pred'], rfiid_eco_alasso_10['pred'], rfiid_sent_rfiid_10['pred'], rfiid_eco_rfiid_10['pred']], axis = 1)
# rfiid_predictions_10.columns = new_names
# rfiid_predictions_10

In [34]:
""" LAG = 22 """

""" RF_IID with no selection"""
rfiid_sent_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection=None, function='RF_IID')
rfiid_eco_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection=None, function='RF_IID')
rfiid_har_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='HAR', selection=None, function='RF_IID')

""" RF_IID with LASSO selection"""
rfiid_sent_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='LASSO', function='RF_IID')
rfiid_eco_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='LASSO', function='RF_IID')
# rfiid_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_IID')

""" RF_IID with ALASSO selection"""
rfiid_sent_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='ALASSO', function='RF_IID')
rfiid_eco_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='ALASSO', function='RF_IID')
# rfiid_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_IID')

""" RF_IID with RF_IID selection"""
rfiid_sent_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_IID', function='RF_IID')
rfiid_eco_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_IID', function='RF_IID')
# rfiid_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_IID')

# """ RF_IID with RF_SB selection"""
# rfiid_sent_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_SB', function='RF_IID')
# rfiid_eco_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_SB', function='RF_IID')
# rfiid_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_IID')

In [95]:
# rfiid_results_22 = pd.concat([rfiid_sent_none_22['errors'], rfiid_eco_none_22['errors'], rfiid_har_none_22['errors'], rfiid_sent_lasso_22['errors'], rfiid_eco_lasso_22['errors'],
#                            rfiid_sent_alasso_22['errors'], rfiid_eco_alasso_22['errors'], rfiid_sent_rfiid_22['errors'], rfiid_eco_rfiid_22['errors'],  rfiid_sent_rfsb_22['errors'],
#                            rfiid_eco_rfsb_22['errors']], axis = 0).transpose()
# rfiid_results_22.columns = new_names
# rfiid_results_22

In [35]:
rfiid_results_22 = pd.concat([rfiid_sent_none_22['errors'], rfiid_eco_none_22['errors'], rfiid_har_none_22['errors'], rfiid_sent_lasso_22['errors'], rfiid_eco_lasso_22['errors'],
                           rfiid_sent_alasso_22['errors'], rfiid_eco_alasso_22['errors'], rfiid_sent_rfiid_22['errors'], rfiid_eco_rfiid_22['errors']], axis = 0).transpose()
rfiid_results_22.columns = new_names
rfiid_results_22

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01506,0.01511,0.03188,0.02227,0.02174,0.01167,0.01140,0.01243,0.01096
mae,0.09535,0.09348,0.14365,0.11597,0.11332,0.08053,0.07910,0.08512,0.07816


In [97]:
# rfiid_predictions_22 = pd.concat([rfiid_sent_none_22['pred'], rfiid_eco_none_22['pred'], rfiid_har_none_22['pred'], rfiid_sent_lasso_22['pred'], rfiid_eco_lasso_22['pred'],
#                            rfiid_sent_alasso_22['pred'], rfiid_eco_alasso_22['pred'], rfiid_sent_rfiid_22['pred'], rfiid_eco_rfiid_22['pred'],  rfiid_sent_rfsb_22['pred'],
#                            rfiid_eco_rfsb_22['pred']], axis = 1)
# rfiid_predictions_22.columns = new_names
# rfiid_predictions_22

In [ ]:
# rfiid_predictions_22 = pd.concat([rfiid_sent_none_22['pred'], rfiid_eco_none_22['pred'], rfiid_har_none_22['pred'], rfiid_sent_lasso_22['pred'], rfiid_eco_lasso_22['pred'],
#                            rfiid_sent_alasso_22['pred'], rfiid_eco_alasso_22['pred'], rfiid_sent_rfiid_22['pred'], rfiid_eco_rfiid_22['pred']], axis = 1)
# rfiid_predictions_22.columns = new_names
# rfiid_predictions_22

#### 3. RF_SB

In [ ]:
""" RF_SB with no selection"""
rfsb_sent_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection=None, function='RF_SB')
rfsb_eco_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection=None, function='RF_SB')
rfsb_har_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection=None, function='RF_SB')

""" RF_SB with LASSO selection"""
rfsb_sent_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='LASSO', function='RF_SB')
rfsb_eco_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='LASSO', function='RF_SB')
# rfsb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_SB')

""" RF_SB with ALASSO selection"""
rfsb_sent_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='ALASSO', function='RF_SB')
rfsb_eco_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='ALASSO', function='RF_SB')
# rfsb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_SB')

""" RF_SB with RF_IID selection"""
rfsb_sent_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_IID', function='RF_SB')
rfsb_eco_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_IID', function='RF_SB')
# rfsb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_SB')

""" RF_SB with RF_SB selection"""
rfsb_sent_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_SB', function='RF_SB')
rfsb_eco_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_SB', function='RF_SB')
# rfsb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_SB')

In [ ]:
rfsb_results_1 = pd.concat([rfsb_sent_none_1['errors'], rfsb_eco_none_1['errors'], rfsb_har_none_1['errors'], rfsb_sent_lasso_1['errors'], rfsb_eco_lasso_1['errors'],
                           rfsb_sent_alasso_1['errors'], rfsb_eco_alasso_1['errors'], rfsb_sent_rfiid_1['errors'], rfsb_eco_rfiid_1['errors'],  rfsb_sent_rfsb_1['errors'],
                           rfsb_eco_rfsb_1['errors']], axis = 0).transpose()
rfsb_results_1.columns = new_names
rfsb_results_1

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid,sent_rfsb,eco_rfsb
rmse,0.00157,0.00800,0.01031,0.00161,0.00873,0.00677,0.01160,0.00094,0.00755,0.00111,0.00773
mae,0.05653,0.06249,0.06199,0.05664,0.06350,0.05996,0.06394,0.05752,0.06436,0.05765,0.06373


In [ ]:
# rfsb_predictions_1 = pd.concat([rfsb_sent_none_1['pred'], rfsb_eco_none_1['pred'], rfsb_har_none_1['pred'], rfsb_sent_lasso_1['pred'], rfsb_eco_lasso_1['pred'],
#                            rfsb_sent_alasso_1['pred'], rfsb_eco_alasso_1['pred'], rfsb_sent_rfiid_1['pred'], rfsb_eco_rfiid_1['pred'],  rfsb_sent_rfsb_1['pred'],
#                            rfsb_eco_rfsb_1['pred']], axis = 1)
# rfsb_predictions_1.columns = new_names
# rfsb_predictions_1

In [ ]:
""" LAG = 5 """

""" RF_SB with no selection"""
rfsb_sent_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection=None, function='RF_SB')
rfsb_eco_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection=None, function='RF_SB')
rfsb_har_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='HAR', selection=None, function='RF_SB')

""" RF_SB with LASSO selection"""
rfsb_sent_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='LASSO', function='RF_SB')
rfsb_eco_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='LASSO', function='RF_SB')
# rfsb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_SB')

""" RF_SB with ALASSO selection"""
rfsb_sent_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='ALASSO', function='RF_SB')
rfsb_eco_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='ALASSO', function='RF_SB')
# rfsb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_SB')

""" RF_SB with RF_IID selection"""
rfsb_sent_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_IID', function='RF_SB')
rfsb_eco_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_IID', function='RF_SB')
# rfsb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_SB')

""" RF_SB with RF_SB selection"""
rfsb_sent_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_SB', function='RF_SB')
rfsb_eco_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_SB', function='RF_SB')
# rfsb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_SB')

In [ ]:
rfsb_results_5 = pd.concat([rfsb_sent_none_5['errors'], rfsb_eco_none_5['errors'], rfsb_har_none_5['errors'], rfsb_sent_lasso_5['errors'], rfsb_eco_lasso_5['errors'],
                           rfsb_sent_alasso_5['errors'], rfsb_eco_alasso_5['errors'], rfsb_sent_rfiid_5['errors'], rfsb_eco_rfiid_5['errors'],  rfsb_sent_rfsb_5['errors'],
                           rfsb_eco_rfsb_5['errors']], axis = 0).transpose()
rfsb_results_5.columns = new_names
rfsb_results_5

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid,sent_rfsb,eco_rfsb
rmse,0.02029,0.02220,0.03128,0.02490,0.02323,0.02806,0.02837,0.01541,0.01865,0.01198,0.01225
mae,0.12021,0.12290,0.14562,0.12318,0.12366,0.12109,0.11740,0.11554,0.11628,0.11736,0.11865


In [ ]:
# rfsb_predictions_5 = pd.concat([rfsb_sent_none_5['pred'], rfsb_eco_none_5['pred'], rfsb_har_none_5['pred'], rfsb_sent_lasso_5['pred'], rfsb_eco_lasso_5['pred'],
#                            rfsb_sent_alasso_5['pred'], rfsb_eco_alasso_5['pred'], rfsb_sent_rfiid_5['pred'], rfsb_eco_rfiid_5['pred'],  rfsb_sent_rfsb_5['pred'],
#                            rfsb_eco_rfsb_5['pred']], axis = 1)
# rfsb_predictions_5.columns = new_names
# rfsb_predictions_5

In [ ]:
""" LAG = 10 """

""" RF_SB with no selection"""
rfsb_sent_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection=None, function='RF_SB')
rfsb_eco_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection=None, function='RF_SB')
rfsb_har_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='HAR', selection=None, function='RF_SB')

""" RF_SB with LASSO selection"""
rfsb_sent_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='LASSO', function='RF_SB')
rfsb_eco_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='LASSO', function='RF_SB')
# rfsb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_SB')

""" RF_SB with ALASSO selection"""
rfsb_sent_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='ALASSO', function='RF_SB')
rfsb_eco_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='ALASSO', function='RF_SB')
# rfsb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_SB')

""" RF_SB with RF_IID selection"""
rfsb_sent_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_IID', function='RF_SB')
rfsb_eco_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_IID', function='RF_SB')
# rfsb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_SB')

""" RF_SB with RF_SB selection"""
rfsb_sent_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_SB', function='RF_SB')
rfsb_eco_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_SB', function='RF_SB')
# rfsb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_SB')

In [ ]:
rfsb_results_10 = pd.concat([rfsb_sent_none_10['errors'], rfsb_eco_none_10['errors'], rfsb_har_none_10['errors'], rfsb_sent_lasso_10['errors'], rfsb_eco_lasso_10['errors'],
                           rfsb_sent_alasso_10['errors'], rfsb_eco_alasso_10['errors'], rfsb_sent_rfiid_10['errors'], rfsb_eco_rfiid_10['errors'],  rfsb_sent_rfsb_10['errors'],
                           rfsb_eco_rfsb_10['errors']], axis = 0).transpose()
rfsb_results_10.columns = new_names
rfsb_results_10

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid,sent_rfsb,eco_rfsb
rmse,0.03396,0.03535,0.04060,0.03927,0.03833,0.04098,0.04213,0.03342,0.03550,0.03011,0.03059
mae,0.12368,0.12501,0.16601,0.12231,0.12090,0.11934,0.12017,0.11698,0.11858,0.11727,0.11725


In [ ]:
# rfsb_predictions_10 = pd.concat([rfsb_sent_none_10['pred'], rfsb_eco_none_10['pred'], rfsb_har_none_10['pred'], rfsb_sent_lasso_10['pred'], rfsb_eco_lasso_10['pred'],
#                            rfsb_sent_alasso_10['pred'], rfsb_eco_alasso_10['pred'], rfsb_sent_rfiid_10['pred'], rfsb_eco_rfiid_10['pred'],  rfsb_sent_rfsb_10['pred'],
#                            rfsb_eco_rfsb_10['pred']], axis = 1)
# rfsb_predictions_10.columns = new_names
# rfsb_predictions_10

In [ ]:
""" LAG = 22 """

""" RF_SB with no selection"""
rfsb_sent_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection=None, function='RF_SB')
rfsb_eco_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection=None, function='RF_SB')
rfsb_har_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='HAR', selection=None, function='RF_SB')

""" RF_SB with LASSO selection"""
rfsb_sent_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='LASSO', function='RF_SB')
rfsb_eco_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='LASSO', function='RF_SB')
# rfsb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='RF_SB')

""" RF_SB with ALASSO selection"""
rfsb_sent_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='ALASSO', function='RF_SB')
rfsb_eco_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='ALASSO', function='RF_SB')
# rfsb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='RF_SB')

""" RF_SB with RF_IID selection"""
rfsb_sent_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_IID', function='RF_SB')
rfsb_eco_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_IID', function='RF_SB')
# rfsb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='RF_SB')

""" RF_SB with RF_SB selection"""
rfsb_sent_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_SB', function='RF_SB')
rfsb_eco_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_SB', function='RF_SB')
# rfsb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='RF_SB')

In [ ]:
rfsb_results_22 = pd.concat([rfsb_sent_none_22['errors'], rfsb_eco_none_22['errors'], rfsb_har_none_22['errors'], rfsb_sent_lasso_22['errors'], rfsb_eco_lasso_22['errors'],
                           rfsb_sent_alasso_22['errors'], rfsb_eco_alasso_22['errors'], rfsb_sent_rfiid_22['errors'], rfsb_eco_rfiid_22['errors'],  rfsb_sent_rfsb_22['errors'],
                           rfsb_eco_rfsb_22['errors']], axis = 0).transpose()
rfsb_results_22.columns = new_names
rfsb_results_22

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid,sent_rfsb,eco_rfsb
rmse,0.04586,0.04600,0.07186,0.03475,0.03481,0.03777,0.03912,0.03956,0.03844,0.03783,0.03836
mae,0.12575,0.12540,0.20003,0.11559,0.11552,0.11414,0.11019,0.11610,0.11619,0.11421,0.11377


In [ ]:
# rfsb_predictions_22 = pd.concat([rfsb_sent_none_22['pred'], rfsb_eco_none_22['pred'], rfsb_har_none_22['pred'], rfsb_sent_lasso_22['pred'], rfsb_eco_lasso_22['pred'],
#                            rfsb_sent_alasso_22['pred'], rfsb_eco_alasso_22['pred'], rfsb_sent_rfiid_22['pred'], rfsb_eco_rfiid_22['pred'],  rfsb_sent_rfsb_22['pred'],
#                            rfsb_eco_rfsb_22['pred']], axis = 1)
# rfsb_predictions_22.columns = new_names
# rfsb_predictions_22

#### 4. XGB

In [37]:
""" XGB with no selection"""
xgb_sent_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection=None, function='XGB')
xgb_eco_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection=None, function='XGB')
xgb_har_none_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection=None, function='XGB')

""" XGB with LASSO selection"""
xgb_sent_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='LASSO', function='XGB')
xgb_eco_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='LASSO', function='XGB')
# xgb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='XGB')

""" XGB with ALASSO selection"""
xgb_sent_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='ALASSO', function='XGB')
xgb_eco_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='ALASSO', function='XGB')
# xgb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='XGB')

""" XGB with RF_IID selection"""
xgb_sent_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_IID', function='XGB')
xgb_eco_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_IID', function='XGB')
# xgb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='XGB')

# """ XGB with RF_SB selection"""
# xgb_sent_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='SENT', selection='RF_SB', function='XGB')
# xgb_eco_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='ECO', selection='RF_SB', function='XGB')
# xgb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='XGB')

In [38]:
# xgb_results_1 = pd.concat([xgb_sent_none_1['errors'], xgb_eco_none_1['errors'], xgb_har_none_1['errors'], xgb_sent_lasso_1['errors'], xgb_eco_lasso_1['errors'],
#                            xgb_sent_alasso_1['errors'], xgb_eco_alasso_1['errors'], xgb_sent_rfiid_1['errors'], xgb_eco_rfiid_1['errors'],  xgb_sent_rfsb_1['errors'],
#                            xgb_eco_rfsb_1['errors']], axis = 0).transpose()
# xgb_results_1.columns = new_names
# xgb_results_1

In [39]:
xgb_results_1 = pd.concat([xgb_sent_none_1['errors'], xgb_eco_none_1['errors'], xgb_har_none_1['errors'], xgb_sent_lasso_1['errors'], xgb_eco_lasso_1['errors'],
                           xgb_sent_alasso_1['errors'], xgb_eco_alasso_1['errors'], xgb_sent_rfiid_1['errors'], xgb_eco_rfiid_1['errors']], axis = 0).transpose()
xgb_results_1.columns = new_names
xgb_results_1

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.00472,0.00743,0.00719,0.00438,0.00738,0.00419,0.00619,0.00412,0.00577
mae,0.05314,0.06589,0.06466,0.05066,0.06502,0.05142,0.06048,0.05074,0.05937


In [27]:
# xgb_predictions_1 = pd.concat([xgb_sent_none_1['pred'], xgb_eco_none_1['pred'], xgb_har_none_1['pred'], xgb_sent_lasso_1['pred'], xgb_eco_lasso_1['pred'],
#                            xgb_sent_alasso_1['pred'], xgb_eco_alasso_1['pred'], xgb_sent_rfiid_1['pred'], xgb_eco_rfiid_1['pred'],  xgb_sent_rfsb_1['pred'],
#                            xgb_eco_rfsb_1['pred']], axis = 1)
# xgb_predictions_1.columns = new_names
# xgb_predictions_1

In [ ]:
# xgb_predictions_1 = pd.concat([xgb_sent_none_1['pred'], xgb_eco_none_1['pred'], xgb_har_none_1['pred'], xgb_sent_lasso_1['pred'], xgb_eco_lasso_1['pred'],
#                            xgb_sent_alasso_1['pred'], xgb_eco_alasso_1['pred'], xgb_sent_rfiid_1['pred'], xgb_eco_rfiid_1['pred']], axis = 1)
# xgb_predictions_1.columns = new_names
# xgb_predictions_1

In [40]:
""" LAG 5 """

""" XGB with no selection"""
xgb_sent_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection=None, function='XGB')
xgb_eco_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection=None, function='XGB')
xgb_har_none_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='HAR', selection=None, function='XGB')

""" XGB with LASSO selection"""
xgb_sent_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='LASSO', function='XGB')
xgb_eco_lasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='LASSO', function='XGB')
# xgb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='XGB')

""" XGB with ALASSO selection"""
xgb_sent_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='ALASSO', function='XGB')
xgb_eco_alasso_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='ALASSO', function='XGB')
# xgb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='XGB')

""" XGB with RF_IID selection"""
xgb_sent_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_IID', function='XGB')
xgb_eco_rfiid_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_IID', function='XGB')
# xgb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='XGB')

# """ XGB with RF_SB selection"""
# xgb_sent_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='SENT', selection='RF_SB', function='XGB')
# xgb_eco_rfsb_5 = fc_rolling_window(Y, npred=120, indice=0, lag=5, method='ECO', selection='RF_SB', function='XGB')
# xgb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='XGB')

In [187]:
# xgb_results_5 = pd.concat([xgb_sent_none_5['errors'], xgb_eco_none_5['errors'], xgb_har_none_5['errors'], xgb_sent_lasso_5['errors'], xgb_eco_lasso_5['errors'],
#                            xgb_sent_alasso_5['errors'], xgb_eco_alasso_5['errors'], xgb_sent_rfiid_5['errors'], xgb_eco_rfiid_5['errors'],  xgb_sent_rfsb_5['errors'],
#                            xgb_eco_rfsb_5['errors']], axis = 0).transpose()
# xgb_results_5.columns = new_names
# xgb_results_5

In [41]:
xgb_results_5 = pd.concat([xgb_sent_none_5['errors'], xgb_eco_none_5['errors'], xgb_har_none_5['errors'], xgb_sent_lasso_5['errors'], xgb_eco_lasso_5['errors'],
                           xgb_sent_alasso_5['errors'], xgb_eco_alasso_5['errors'], xgb_sent_rfiid_5['errors'], xgb_eco_rfiid_5['errors']], axis = 0).transpose()
xgb_results_5.columns = new_names
xgb_results_5

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01109,0.01307,0.02308,0.01149,0.01224,0.00901,0.00839,0.00707,0.00675
mae,0.08117,0.08474,0.12061,0.08430,0.08433,0.07010,0.07040,0.06589,0.06282


In [189]:
# xgb_predictions_5 = pd.concat([xgb_sent_none_5['pred'], xgb_eco_none_5['pred'], xgb_har_none_5['pred'], xgb_sent_lasso_5['pred'], xgb_eco_lasso_5['pred'],
#                            xgb_sent_alasso_5['pred'], xgb_eco_alasso_5['pred'], xgb_sent_rfiid_5['pred'], xgb_eco_rfiid_5['pred'],  xgb_sent_rfsb_5['pred'],
#                            xgb_eco_rfsb_5['pred']], axis = 1)
# xgb_predictions_5.columns = new_names
# xgb_predictions_5

In [ ]:
# xgb_predictions_5 = pd.concat([xgb_sent_none_5['pred'], xgb_eco_none_5['pred'], xgb_har_none_5['pred'], xgb_sent_lasso_5['pred'], xgb_eco_lasso_5['pred'],
#                            xgb_sent_alasso_5['pred'], xgb_eco_alasso_5['pred'], xgb_sent_rfiid_5['pred'], xgb_eco_rfiid_5['pred']], axis = 1)
# xgb_predictions_5.columns = new_names
# xgb_predictions_5

In [42]:
""" LAG 10 """

""" XGB with no selection"""
xgb_sent_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection=None, function='XGB')
xgb_eco_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection=None, function='XGB')
xgb_har_none_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='HAR', selection=None, function='XGB')

""" XGB with LASSO selection"""
xgb_sent_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='LASSO', function='XGB')
xgb_eco_lasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='LASSO', function='XGB')
# xgb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='XGB')

""" XGB with ALASSO selection"""
xgb_sent_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='ALASSO', function='XGB')
xgb_eco_alasso_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='ALASSO', function='XGB')
# xgb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='XGB')

""" XGB with RF_IID selection"""
xgb_sent_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_IID', function='XGB')
xgb_eco_rfiid_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_IID', function='XGB')
# xgb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='XGB')

# """ XGB with RF_SB selection"""
# xgb_sent_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='SENT', selection='RF_SB', function='XGB')
# xgb_eco_rfsb_10 = fc_rolling_window(Y, npred=120, indice=0, lag=10, method='ECO', selection='RF_SB', function='XGB')
# xgb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='XGB')

In [192]:
# xgb_results_10 = pd.concat([xgb_sent_none_10['errors'], xgb_eco_none_10['errors'], xgb_har_none_10['errors'], xgb_sent_lasso_10['errors'], xgb_eco_lasso_10['errors'],
#                            xgb_sent_alasso_10['errors'], xgb_eco_alasso_10['errors'], xgb_sent_rfiid_10['errors'], xgb_eco_rfiid_10['errors'],  xgb_sent_rfsb_10['errors'],
#                            xgb_eco_rfsb_10['errors']], axis = 0).transpose()
# xgb_results_10.columns = new_names
# xgb_results_10

In [43]:
xgb_results_10 = pd.concat([xgb_sent_none_10['errors'], xgb_eco_none_10['errors'], xgb_har_none_10['errors'], xgb_sent_lasso_10['errors'], xgb_eco_lasso_10['errors'],
                           xgb_sent_alasso_10['errors'], xgb_eco_alasso_10['errors'], xgb_sent_rfiid_10['errors'], xgb_eco_rfiid_10['errors']], axis = 0).transpose()
xgb_results_10.columns = new_names
xgb_results_10

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01541,0.01677,0.03224,0.01775,0.01844,0.01160,0.01347,0.01074,0.01202
mae,0.09672,0.09973,0.14379,0.09370,0.09985,0.08320,0.08284,0.07906,0.07860


In [194]:
# xgb_predictions_10 = pd.concat([xgb_sent_none_10['pred'], xgb_eco_none_10['pred'], xgb_har_none_10['pred'], xgb_sent_lasso_10['pred'], xgb_eco_lasso_10['pred'],
#                            xgb_sent_alasso_10['pred'], xgb_eco_alasso_10['pred'], xgb_sent_rfiid_10['pred'], xgb_eco_rfiid_10['pred'],  xgb_sent_rfsb_10['pred'],
#                            xgb_eco_rfsb_10['pred']], axis = 1)
# xgb_predictions_10.columns = new_names
# xgb_predictions_10

In [ ]:
# xgb_predictions_10 = pd.concat([xgb_sent_none_10['pred'], xgb_eco_none_10['pred'], xgb_har_none_10['pred'], xgb_sent_lasso_10['pred'], xgb_eco_lasso_10['pred'],
#                            xgb_sent_alasso_10['pred'], xgb_eco_alasso_10['pred'], xgb_sent_rfiid_10['pred'], xgb_eco_rfiid_10['pred']], axis = 1)
# xgb_predictions_10.columns = new_names
# xgb_predictions_10

In [44]:
""" LAG 22 """

""" XGB with no selection"""
xgb_sent_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection=None, function='XGB')
xgb_eco_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection=None, function='XGB')
xgb_har_none_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='HAR', selection=None, function='XGB')

""" XGB with LASSO selection"""
xgb_sent_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='LASSO', function='XGB')
xgb_eco_lasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='LASSO', function='XGB')
# xgb_har_lasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='LASSO', function='XGB')

""" XGB with ALASSO selection"""
xgb_sent_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='ALASSO', function='XGB')
xgb_eco_alasso_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='ALASSO', function='XGB')
# xgb_har_alasso_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='ALASSO', function='XGB')

""" XGB with RF_IID selection"""
xgb_sent_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_IID', function='XGB')
xgb_eco_rfiid_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_IID', function='XGB')
# xgb_har_rfiid_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_IID', function='XGB')

# """ XGB with RF_SB selection"""
# xgb_sent_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='SENT', selection='RF_SB', function='XGB')
# xgb_eco_rfsb_22 = fc_rolling_window(Y, npred=120, indice=0, lag=22, method='ECO', selection='RF_SB', function='XGB')
# xgb_har_rfsb_1 = fc_rolling_window(Y, npred=120, indice=0, lag=1, method='HAR', selection='RF_SB', function='XGB')

In [197]:
# xgb_results_22 = pd.concat([xgb_sent_none_22['errors'], xgb_eco_none_22['errors'], xgb_har_none_22['errors'], xgb_sent_lasso_22['errors'], xgb_eco_lasso_22['errors'],
#                            xgb_sent_alasso_22['errors'], xgb_eco_alasso_22['errors'], xgb_sent_rfiid_22['errors'], xgb_eco_rfiid_22['errors'],  xgb_sent_rfsb_22['errors'],
#                            xgb_eco_rfsb_22['errors']], axis = 0).transpose()
# xgb_results_22.columns = new_names
# xgb_results_22

In [45]:
xgb_results_22 = pd.concat([xgb_sent_none_22['errors'], xgb_eco_none_22['errors'], xgb_har_none_22['errors'], xgb_sent_lasso_22['errors'], xgb_eco_lasso_22['errors'],
                           xgb_sent_alasso_22['errors'], xgb_eco_alasso_22['errors'], xgb_sent_rfiid_22['errors'], xgb_eco_rfiid_22['errors']], axis = 0).transpose()
xgb_results_22.columns = new_names
xgb_results_22

,sent_none,eco_none,har_none,sent_lasso,eco_lasso,sent_alasso,eco_alasso,sent_rfiid,eco_rfiid
mse,0.01214,0.01012,0.03708,0.02458,0.02475,0.00821,0.00819,0.00837,0.00707
mae,0.08060,0.07623,0.15607,0.11859,0.11649,0.07090,0.06962,0.07039,0.06560


In [199]:
# xgb_predictions_22 = pd.concat([xgb_sent_none_22['pred'], xgb_eco_none_22['pred'], xgb_har_none_22['pred'], xgb_sent_lasso_22['pred'], xgb_eco_lasso_22['pred'],
#                            xgb_sent_alasso_22['pred'], xgb_eco_alasso_22['pred'], xgb_sent_rfiid_22['pred'], xgb_eco_rfiid_22['pred'],  xgb_sent_rfsb_22['pred'],
#                            xgb_eco_rfsb_22['pred']], axis = 1)
# xgb_predictions_22.columns = new_names
# xgb_predictions_22

In [ ]:
# xgb_predictions_22 = pd.concat([xgb_sent_none_22['pred'], xgb_eco_none_22['pred'], xgb_har_none_22['pred'], xgb_sent_lasso_22['pred'], xgb_eco_lasso_22['pred'],
#                            xgb_sent_alasso_22['pred'], xgb_eco_alasso_22['pred'], xgb_sent_rfiid_22['pred'], xgb_eco_rfiid_22['pred']], axis = 1)
# xgb_predictions_22.columns = new_names
# xgb_predictions_22

## Results

#### 1. Forecast Horizon = 1

In [46]:
# final_name = ['ols_rmse','ols_mae','rfiid_rmse','rfiid_mae','rfsb_rmse','rfsb_mae','xgb_rmse','xgb_mae']
final_name = ['ols_mse','ols_mae','rfiid_mse','rfiid_mae','xgb_mse','xgb_mae']

In [47]:
# final_results_1 = pd.concat([ols_results_1, rfiid_results_1, rfsb_results_1, xgb_results_1], axis = 0).transpose()
# final_results_1.columns = final_name
# final_results_1

final_results_1 = pd.concat([ols_results_1, rfiid_results_1, xgb_results_1], axis = 0).transpose()
final_results_1.columns = final_name
final_results_1

,ols_mse,ols_mae,rfiid_mse,rfiid_mae,xgb_mse,xgb_mae
sent_none,0.00965,0.07856,0.00419,0.04923,0.00472,0.05314
eco_none,0.01049,0.08291,0.00543,0.05516,0.00743,0.06589
har_none,0.00492,0.05043,0.00627,0.05891,0.00719,0.06466
sent_lasso,0.00452,0.04945,0.00343,0.04423,0.00438,0.05066
eco_lasso,0.00555,0.05711,0.00527,0.05613,0.00738,0.06502
sent_alasso,0.00452,0.05039,0.00353,0.04536,0.00419,0.05142
eco_alasso,0.00543,0.05611,0.00539,0.05559,0.00619,0.06048
sent_rfiid,0.00459,0.04884,0.00354,0.04691,0.00412,0.05074
eco_rfiid,0.00551,0.05688,0.00579,0.05769,0.00577,0.05937


In [48]:
# final_results_1.to_csv("../2024 updates/final_results_1_sample2.csv")

#### 2. Forecast Horizon = 5

In [49]:
# final_results_5 = pd.concat([ols_results_5, rfiid_results_5, rfsb_results_5, xgb_results_5], axis = 0).transpose()
final_results_5 = pd.concat([ols_results_5, rfiid_results_5, xgb_results_5], axis = 0).transpose()
final_results_5.columns = final_name
final_results_5

,ols_mse,ols_mae,rfiid_mse,rfiid_mae,xgb_mse,xgb_mae
sent_none,0.03706,0.14315,0.01121,0.08532,0.01109,0.08117
eco_none,0.03688,0.14936,0.01103,0.08478,0.01307,0.08474
har_none,0.01802,0.10113,0.02153,0.11488,0.02308,0.12061
sent_lasso,0.02644,0.12896,0.01237,0.08768,0.01149,0.08430
eco_lasso,0.02466,0.12829,0.01237,0.08654,0.01224,0.08433
sent_alasso,0.02159,0.11524,0.01030,0.08010,0.00901,0.07010
eco_alasso,0.02346,0.12194,0.01021,0.07925,0.00839,0.07040
sent_rfiid,0.02158,0.10817,0.00919,0.07331,0.00707,0.06589
eco_rfiid,0.02015,0.11553,0.00876,0.07311,0.00675,0.06282


In [50]:
# final_results_5 .to_csv("../2024 updates/final_results_5_sample2.csv")

#### 3. Forecast Horizon = 10

In [51]:
# final_results_10 = pd.concat([ols_results_10, rfiid_results_10, rfsb_results_10, xgb_results_10], axis = 0).transpose()
final_results_10 = pd.concat([ols_results_10, rfiid_results_10, xgb_results_10], axis = 0).transpose()
final_results_10.columns = final_name
final_results_10

,ols_mse,ols_mae,rfiid_mse,rfiid_mae,xgb_mse,xgb_mae
sent_none,0.05643,0.18964,0.01623,0.10032,0.01541,0.09672
eco_none,0.04591,0.17286,0.01739,0.10165,0.01677,0.09973
har_none,0.02874,0.13266,0.02938,0.13938,0.03224,0.14379
sent_lasso,0.04074,0.16258,0.01400,0.09279,0.01775,0.09370
eco_lasso,0.03752,0.15801,0.01622,0.09863,0.01844,0.09985
sent_alasso,0.03584,0.15585,0.01293,0.08690,0.01160,0.08320
eco_alasso,0.03930,0.16157,0.01284,0.08739,0.01347,0.08284
sent_rfiid,0.04077,0.17392,0.01215,0.08426,0.01074,0.07906
eco_rfiid,0.03587,0.16154,0.01282,0.08401,0.01202,0.07860


In [52]:
# final_results_10 .to_csv("../2024 updates/final_results_10_sample2.csv")

#### 4. Forecast Horizon = 22

In [53]:
# final_results_22 = pd.concat([ols_results_22, rfiid_results_22, rfsb_results_22, xgb_results_22], axis = 0).transpose()
final_results_22 = pd.concat([ols_results_22, rfiid_results_22, xgb_results_22], axis = 0).transpose()
final_results_22.columns = final_name
final_results_22

,ols_mse,ols_mae,rfiid_mse,rfiid_mae,xgb_mse,xgb_mae
sent_none,0.05499,0.18056,0.01506,0.09535,0.01214,0.08060
eco_none,0.04631,0.16749,0.01511,0.09348,0.01012,0.07623
har_none,0.03581,0.15662,0.03188,0.14365,0.03708,0.15607
sent_lasso,0.03334,0.14748,0.02227,0.11597,0.02458,0.11859
eco_lasso,0.03191,0.15023,0.02174,0.11332,0.02475,0.11649
sent_alasso,0.03336,0.14983,0.01167,0.08053,0.00821,0.07090
eco_alasso,0.03094,0.14668,0.01140,0.07910,0.00819,0.06962
sent_rfiid,0.07822,0.22877,0.01243,0.08512,0.00837,0.07039
eco_rfiid,0.07530,0.21538,0.01096,0.07816,0.00707,0.06560


In [54]:
# final_results_22 .to_csv("../2024 updates/final_results_22_sample2.csv")

In [55]:
# writer_0 = pd.ExcelWriter('../2024 updates/errors_sample2_version_1.xlsx', engine='xlsxwriter')

# final_results_1.to_excel(writer_0, sheet_name='horizon_1')
# final_results_5.to_excel(writer_0, sheet_name='horizon_5')
# final_results_10.to_excel(writer_0, sheet_name='horizon_10')
# final_results_22.to_excel(writer_0, sheet_name='horizon_22')

# writer_0.save()